# Physically Recurrent Neural Networks - Demo notebook

This notebook demonstrates how to train a PRNN (or an RNN), save and load checkpoints, and evaluate model performance after training.

## How to use this resource

After loading packages and downloading checkpoints and datasets with the first block, all other code blocks are self-contained. They perform a range of different tasks involving training, validating and comparing PRNNs against each other and against variational RNNs. Running the notebook from top to bottom is therefore not necessary. Feel free to use these blocks as the starting point of more involved applications of PRNNs.

## Load packages, get datasets and checkpoints

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

# If using Google Colab, uncomment the lines below
# Be aware that performance will take a hit when using Colab
    
# !pip install ipympl==0.9.3
# !pip install matplotlib==3.5.2
# from google.colab import output
# output.enable_custom_widget_manager()
# %cd '/content/drive/MyDrive/your_folder'

import os
import random
import numpy as np
import torch
import glob
import shutil
import zipfile

import matplotlib.pyplot as plt

from urllib.request import urlretrieve

from utils import Trainer
from utils import StressStrainDataset

from prnn import PRNN
from rnn  import GRU
from rnn  import LSTM
from rnn  import ELBOLoss

from visualizer import PlotNN

torch.set_default_dtype(torch.float64)

# Download and unzip datasets and checkpoints, if necessary.
# The files should take in total about 18mb of space.
# Comment these lines if that is not desirable
if not os.path.isdir('datasets'):
    print('Downloading and unzipping datasets...')
    urlretrieve('https://surfdrive.surf.nl/files/index.php/s/OcSDq0zNqkVbvIO/download', 'datasets.zip')
    zip_file = zipfile.ZipFile('datasets.zip')
    zip_file.extractall('.')
    
if not os.path.isdir('trained_models'):
    print('Downloading and unzipping model checkpoints...')
    urlretrieve('https://surfdrive.surf.nl/files/index.php/s/XyIWSNUKp47jnrY/download', 'checkpoints.zip')
    zip_file = zipfile.ZipFile('checkpoints.zip')
    zip_file.extractall('.')

## Example code

### Train an RNN on 80 GP paths, for demonstration

In [ ]:
torch.manual_seed(42)

# Data is loaded from a text file in the format [eps_xx eps_yy gam_xy sig_xx sig_yy tau_xy]
# Strain paths are separated by blank lines. All paths have 60 time steps
dataset = StressStrainDataset('datasets/gpCurves.data', [0,1,2], [3,4,5], seq_length=60)

# Split dataset into 80 curves for training and 20 for validation
# manual_seed=42 is also used for pre-trained models
tset, vset = torch.utils.data.random_split(dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(42))

tloader = torch.utils.data.DataLoader(tset, batch_size=4, shuffle=True)
vloader = torch.utils.data.DataLoader(vset, batch_size=20, shuffle=False)

# Initialize network
gru = GRU(n_features=3, n_outputs=3, n_latents=64, dropout=True)

# Train network for very small number of epochs 
# Set learning rate dangerously high to get fast training for demonstration purposes
trainer = Trainer(gru, loss=ELBOLoss(gru.dropout), optimizer=torch.optim.Adam(gru.parameters(), lr=1e-2))
trainer.train(tloader, vloader, epochs=200, patience=10, interval=10)

### Train a PRNN model on 18 simple paths, for demonstration

In [ ]:
torch.manual_seed(42)

dataset = StressStrainDataset('datasets/canonical.data', [0,1,2], [3,4,5], seq_length=60)

tloader = torch.utils.data.DataLoader(dataset, batch_size=3, shuffle=True)

prnn = PRNN(n_features=3, n_outputs=3, n_matpts=2)

# Set learning rate dangerously high to get fast training for demonstration purposes
trainer = Trainer(prnn, optimizer=torch.optim.Adam(prnn.parameters(), lr=1e-1))

# Use training set for validation (effectively disabling early stopping)
trainer.train(tloader, tloader, epochs=20, patience=10)

# Save the partially trained model
trainer.save('trained_models/my_first_prnn.pth')

### Load a pre-trained PRNN and use it to evaluate 20 GP curves

In [ ]:
torch.manual_seed(42)

dataset = StressStrainDataset('datasets/gpCurves.data', [0,1,2], [3,4,5], seq_length=60)

_, vset = torch.utils.data.random_split(dataset, [0.80, 0.20], generator=torch.Generator().manual_seed(42))

vloader = torch.utils.data.DataLoader(vset, batch_size=1, shuffle=False) # Set batch_size=1 so error is printed for each strain path

prnn_pre = PRNN(n_features=3, n_outputs=3, n_matpts=2)
trainer_pre = Trainer(prnn_pre)

trainer_pre.load('trained_models/prnn_gp_80_0.pth') # Trained on 80 GP curves for 10000 epochs

trainer_pre.eval(vloader, loss=torch.nn.L1Loss()) # Absolute error (L1Loss) for better interpretability of results

### Compare two different models curve by curve

In [ ]:
torch.manual_seed(42)

dataset = StressStrainDataset('datasets/gpCurves.data', [0,1,2], [3,4,5], seq_length=60)

_, vset = torch.utils.data.random_split(dataset, [0.80, 0.20], generator=torch.Generator().manual_seed(42))

vloader = torch.utils.data.DataLoader(vset, batch_size=1, shuffle=False)

prnn = PRNN(n_features=3, n_outputs=3, n_matpts=2)

gru  = GRU(n_features=3, n_outputs=3, n_latents=64, dropout=True)

trainer1 = Trainer(prnn)
trainer1.load('trained_models/prnn_gp_5_0.pth') # Trained on 5 GP curves for 10000 epochs

trainer2 = Trainer(gru)
trainer2.load('trained_models/gru_gp_5_0.pth') # Trained on 5 GP curves for 10000 epochs

plot = PlotNN(vloader, [prnn, gru], ['PRNN (5 paths)', 'GRU (5 paths)']) # Try it out for more than 2 models!
plot.add_buttons('previous','random','next')
plot.show()

### Check how prediction accuracy increases with more data (learning curve) with pretrained PRNNs

In [ ]:
torch.manual_seed(42)

test_dataset = StressStrainDataset('datasets/gpCurves.data', [0,1,2], [3,4,5], seq_length=60)

_, vset = torch.utils.data.random_split(test_dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(42))

vloader = torch.utils.data.DataLoader(vset, batch_size=20, shuffle=False) # Set batch_size=20 for efficiency

prnn = PRNN(n_features=3, n_outputs=3, n_matpts=2)

trainer = Trainer(prnn)
networks = glob.glob('trained_models/prnn_gp*.pth')

size = []
loss = []

for fn in networks:
    print('testing network from checkpoint: ' + fn)
    trainer.load(fn)
    size.append(float(fn.split('_')[-2]))
    loss.append(trainer.eval(vloader, loss=torch.nn.L1Loss(),verbose=False))

plt.figure()
plt.plot(size,loss,'.')
plt.title('PRNN learning curve (trained and tested on GP paths)')
plt.ylabel('Loss [MPa]')
plt.xlabel('Training dataset size [strain paths]')
plt.show()

### Plot a GRU learning curve on the same dataset, for comparison

In [ ]:
test_dataset = StressStrainDataset('datasets/gpCurves.data', [0,1,2], [3,4,5], seq_length=60)

_, vset = torch.utils.data.random_split(test_dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(42))

vloader = torch.utils.data.DataLoader(vset, batch_size=20, shuffle=False) # Set batch_size=20 for efficiency

gru = GRU(n_features=3, n_outputs=3, n_latents=64, dropout=True)

trainer = Trainer(gru)
networks = glob.glob('trained_models/gru_gp*.pth')

size = []
loss = []

for fn in networks:
    print('testing network from checkpoint: ' + fn)
    trainer.load(fn)
    size.append(float(fn.split('_')[-2]))
    loss.append(trainer.eval(vloader, loss=torch.nn.L1Loss(), verbose=False))

plt.figure()
plt.plot(size,loss,'.')
plt.title('GRU learning curve (trained and tested on GP paths)')
plt.ylabel('Loss [MPa]')
plt.xlabel('Training dataset size [strain paths]')
plt.show()

### Train a hundred PRNNs from scratch and plot a learning curve

Please note this block can take a long time to run!

In [ ]:
torch.manual_seed(42)

dataset = StressStrainDataset('datasets/gpCurves.data', [0,1,2], [3,4,5], seq_length=60)

Tset, vset = torch.utils.data.random_split(dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(42))

vloader = torch.utils.data.DataLoader(vset, batch_size=1, shuffle=False)

ncurves = list(range(1,11))
nmodels = 10
nepochs = 5000
path    = 'lc_prnn_gp'

# Train 10 models for each dataset size.
# Datasets are uniformly sampled from a pool of 80 curves, i.e
# each of the 10 models has a different training dataset.
# Training and evaluation output suppressed by the 'verbose' flag

if os.path.isdir(path):
    shutil.rmtree(path)
os.mkdir(path)

for ncurve in ncurves:
    for nmodel in range(nmodels):
        tset, xset = torch.utils.data.random_split(Tset, [float(ncurve/80), float(1-ncurve/80)], generator=torch.Generator().manual_seed(ncurve + nmodel))

        tloader = torch.utils.data.DataLoader(tset, batch_size=5, shuffle=True)

        print('\nTraining with',ncurve,'curve(s). Network',nmodel+1,'of',nmodels)

        prnn = PRNN(n_features=3, n_outputs=3, n_matpts=2)
        trainer = Trainer(prnn,optimizer=torch.optim.Adam(prnn.parameters(),lr=1.e-2))
        trainer.train(tloader, vloader, epochs=nepochs, patience=200, interval=10, verbose=False)
        trainer.save(os.path.join(path,'prnn'+'_gp_'+str(ncurve)+'_'+str(nmodel)+'.pth'))

# Plot the learning curve

prnn = PRNN(n_features=3, n_outputs=3, n_matpts=2)

trainer = Trainer(prnn)
networks = glob.glob(path+'/prnn_gp*.pth')

size = []
loss = []

for fn in networks:
    print('\nTesting network from checkpoint: ' + fn)
    trainer.load(fn)
    size.append(float(fn.split('_')[-2]))
    loss.append(trainer.eval(vloader, loss=torch.nn.L1Loss(),verbose=False))

plt.figure()
plt.plot(size,loss,'.')
plt.ylabel('Loss [MPa]')
plt.xlabel('n curves')
plt.show()